<a href="https://colab.research.google.com/github/id-shiv/knowledge_base/blob/master/%5BProject_103%5D_Text_Summarization_with_SpaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [0]:
# Build a List of Stopwords
stopwords = list(STOP_WORDS)

In [0]:
document = "Enhance visibility into the health of your Dell EMC hardware \
environment Dell EMC Server and Network switch, Storage and Printer Management \
Packs for Microsoft System Center Operations Manager (SCOM) deliver greater \
visibility into hardware inventory details, components and configurations \
essential to optimizing the availability of your infrastructure hardware.\
Dell EMC Server Management Pack Suite offers both in-band and agent-free \
options that enable System Center Operations Manager to discover, monitor and \
accurately depict the status of Dell PowerEdge servers and modular platforms, \
integrated Dell Remote Access Controllers (iDRACs) and OpenManage \
Enterprise–Modular (OME-Modular), Chassis Management Controllers (CMCs) on a \
defined network segment.\
Agent-free out-of-band monitoring is available for 12th generation through \
current platforms Dell EMC PowerEdge servers and agent-based monitoring with \
OpenManage Server Administrator (OMSA) is available for 11th generation through \
current platforms PowerEdge servers and PowerVault NX storage appliances.\
OpenManage Integration for Microsoft System Center (OMIMSSC) for System Center \
Operations Manager (SCOM) is available to deploy as a virtual appliance for \
monitoring in SCOM environments for agent-free management."

In [0]:
nlp = spacy.load('en')
docx = nlp(document)

In [0]:
mytokens = [token.text for token in docx]

In [41]:
word_frequencies = {}
for word in docx:
    if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
word_frequencies

{'(': 7,
 ')': 7,
 ',': 5,
 '-': 8,
 '.': 4,
 '11th': 1,
 '12th': 1,
 'Access': 1,
 'Administrator': 1,
 'Agent': 1,
 'CMCs': 1,
 'Center': 4,
 'Chassis': 1,
 'Controllers': 2,
 'Dell': 6,
 'EMC': 4,
 'Enhance': 1,
 'Enterprise': 1,
 'Integration': 1,
 'Management': 3,
 'Manager': 3,
 'Microsoft': 2,
 'Modular': 2,
 'NX': 1,
 'Network': 1,
 'OME': 1,
 'OMIMSSC': 1,
 'OMSA': 1,
 'OpenManage': 3,
 'Operations': 3,
 'Pack': 1,
 'Packs': 1,
 'PowerEdge': 3,
 'PowerVault': 1,
 'Printer': 1,
 'Remote': 1,
 'SCOM': 3,
 'Server': 3,
 'Storage': 1,
 'Suite': 1,
 'System': 4,
 'accurately': 1,
 'agent': 3,
 'appliance': 1,
 'appliances': 1,
 'availability': 1,
 'available': 3,
 'band': 2,
 'based': 1,
 'components': 1,
 'configurations': 1,
 'current': 2,
 'defined': 1,
 'deliver': 1,
 'depict': 1,
 'deploy': 1,
 'details': 1,
 'discover': 1,
 'enable': 1,
 'environment': 1,
 'environments': 1,
 'essential': 1,
 'free': 3,
 'generation': 2,
 'greater': 1,
 'hardware': 3,
 'health': 1,
 'iDRACs':

In [0]:
maximum_frequency = max(word_frequencies.values())

In [0]:
for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

In [44]:
word_frequencies

{'(': 0.875,
 ')': 0.875,
 ',': 0.625,
 '-': 1.0,
 '.': 0.5,
 '11th': 0.125,
 '12th': 0.125,
 'Access': 0.125,
 'Administrator': 0.125,
 'Agent': 0.125,
 'CMCs': 0.125,
 'Center': 0.5,
 'Chassis': 0.125,
 'Controllers': 0.25,
 'Dell': 0.75,
 'EMC': 0.5,
 'Enhance': 0.125,
 'Enterprise': 0.125,
 'Integration': 0.125,
 'Management': 0.375,
 'Manager': 0.375,
 'Microsoft': 0.25,
 'Modular': 0.25,
 'NX': 0.125,
 'Network': 0.125,
 'OME': 0.125,
 'OMIMSSC': 0.125,
 'OMSA': 0.125,
 'OpenManage': 0.375,
 'Operations': 0.375,
 'Pack': 0.125,
 'Packs': 0.125,
 'PowerEdge': 0.375,
 'PowerVault': 0.125,
 'Printer': 0.125,
 'Remote': 0.125,
 'SCOM': 0.375,
 'Server': 0.375,
 'Storage': 0.125,
 'Suite': 0.125,
 'System': 0.5,
 'accurately': 0.125,
 'agent': 0.375,
 'appliance': 0.125,
 'appliances': 0.125,
 'availability': 0.125,
 'available': 0.375,
 'band': 0.25,
 'based': 0.125,
 'components': 0.125,
 'configurations': 0.125,
 'current': 0.25,
 'defined': 0.125,
 'deliver': 0.125,
 'depict': 0.1

In [0]:
sentence_list = [sentence for sentence in docx.sents]

In [46]:
[w.text.lower() for t in sentence_list for w in t ]

['enhance',
 'visibility',
 'into',
 'the',
 'health',
 'of',
 'your',
 'dell',
 'emc',
 'hardware',
 'environment',
 'dell',
 'emc',
 'server',
 'and',
 'network',
 'switch',
 ',',
 'storage',
 'and',
 'printer',
 'management',
 'packs',
 'for',
 'microsoft',
 'system',
 'center',
 'operations',
 'manager',
 '(',
 'scom',
 ')',
 'deliver',
 'greater',
 'visibility',
 'into',
 'hardware',
 'inventory',
 'details',
 ',',
 'components',
 'and',
 'configurations',
 'essential',
 'to',
 'optimizing',
 'the',
 'availability',
 'of',
 'your',
 'infrastructure',
 'hardware',
 '.',
 'dell',
 'emc',
 'server',
 'management',
 'pack',
 'suite',
 'offers',
 'both',
 'in',
 '-',
 'band',
 'and',
 'agent',
 '-',
 'free',
 'options',
 'that',
 'enable',
 'system',
 'center',
 'operations',
 'manager',
 'to',
 'discover',
 ',',
 'monitor',
 'and',
 'accurately',
 'depict',
 'the',
 'status',
 'of',
 'dell',
 'poweredge',
 'servers',
 'and',
 'modular',
 'platforms',
 ',',
 'integrated',
 'dell',
 're

In [0]:
# Sentence Score via comparrng each word with sentence
sentence_scores = {}  
for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

In [0]:
# Sentence Score via comparrng each word with sentence
# Alternative Method
lowered_sentence_list = [w.text.lower() for t in sentence_list for w in t ]
lowered_sentence_scores = {}  
for sent in lowered_sentence_list:  
        for word in sent:
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        lowered_sentence_scores[sent] = word_frequencies[word]
                    else:
                        lowered_sentence_scores[sent] += word_frequencies[word]

In [49]:
# Sentence Score Table
sentence_scores

{OpenManage Integration for Microsoft System Center (OMIMSSC) for System Center Operations Manager (SCOM) is available to deploy as a virtual appliance for monitoring in SCOM environments for agent-free management.: 7.125}

In [0]:
from heapq import nlargest

In [51]:
summarized_sentences = nlargest(7, sentence_scores, key=sentence_scores.get)
summarized_sentences

[OpenManage Integration for Microsoft System Center (OMIMSSC) for System Center Operations Manager (SCOM) is available to deploy as a virtual appliance for monitoring in SCOM environments for agent-free management.]

In [52]:
# Convert Sentences from Spacy Span to Strings for joining entire sentence
for w in summarized_sentences:
    print(w.text)

OpenManage Integration for Microsoft System Center (OMIMSSC) for System Center Operations Manager (SCOM) is available to deploy as a virtual appliance for monitoring in SCOM environments for agent-free management.


In [0]:
# List Comprehension of Sentences Converted From Spacy.span to strings
final_sentences = [ w.text for w in summarized_sentences ]

In [54]:
summary = ' '.join(final_sentences)
summary

'OpenManage Integration for Microsoft System Center (OMIMSSC) for System Center Operations Manager (SCOM) is available to deploy as a virtual appliance for monitoring in SCOM environments for agent-free management.'

In [55]:
print(len(summary))
print(len(document))

213
1277


# Function

In [0]:
def text_summarizer(raw_docx):
    raw_text = raw_docx
    docx = nlp(raw_text)
    stopwords = list(STOP_WORDS)
    # Build Word Frequency
    # word.text is tokenization in spacy
    word_frequencies = {}  
    for word in docx:  
        if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    # Sentence Tokens
    sentence_list = [ sentence for sentence in docx.sents ]

    # Calculate Sentence Score and Ranking
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

    # Find N Largest
    summary_sentences = nlargest(7, sentence_scores, key=sentence_scores.get)
    final_sentences = [ w.text for w in summary_sentences ]
    summary = ' '.join(final_sentences)
    print("Original Document\n")
    print(raw_docx)
    print("Total Length:",len(raw_docx))
    print('\n\nSummarized Document\n')
    print(summary)
    print("Total Length:",len(summary))

In [90]:
import bs4
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
from nltk.tokenize import RegexpTokenizer

url = 'https://en.wikipedia.org/wiki/Dell'
url = 'https://www.dell.com/en-in/work/shop/povw/openmanage-microsoft'
web_page = urlopen(url=url)
html_page = web_page.read()
page = soup(html_page, 'html.parser')
whitelist = [
  'p'
]

text_elements = [t for t in page.find_all(text=True) if t.parent.name in whitelist]

document = " ".join([word for word in text_elements if word not in ['\n', ' , ']])
document = document.replace("  ,", "").replace("  ", "")
# tokenizer = RegexpTokenizer(r'\w+')
# result = tokenizer.tokenize(document)
# document = " ".join([word for word in result])

text_summarizer(document)

Original Document

[if !IE]> <br class="hidden-xs"/> <![endif] [if !IE]> <br class="hidden-xs"/> <![endif] [if !IE]> <br class="hidden-xs"/> <![endif] Boost the operational efficiency of Dell EMC hardware using OpenManage Integration for Microsoft System Center and your existing Microsoft System Center systems management solutions.deliver greater visibility into hardware inventory details, components and configurations essential to optimizing the availability of your infrastructure hardware.  offers both in-band and agent-free options that enable System Center Operations Manager to discover, monitor and accurately depict the status of Dell PowerEdge servers and modular platforms, integrated Dell Remote Access Controllers (iDRACs) and OpenManage Enterprise–Modular (OME-Modular), Chassis Management Controllers (CMCs) on a defined network segment. Agent-free out-of-band monitoring is available for 12th generation through current platforms Dell EMC PowerEdge servers and agent-based monitor